# 실전 문제 해결 (과적합)

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 100

## 모델 정의

In [3]:
class ConvLNReluBlock(tf.keras.Model):  # Layer Normalization
  def __init__(self, filters, kernel_size):
    super(ConvLNReluBlock, self).__init__()
    self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', use_bias=False)
    self.ln = tf.keras.layers.LayerNormalization()  # activation은 사용하지 않는다. layer normalazation 이후에 사용.

  def call(self, x, training=False, mask=None):
    x = self.conv(x)
    x = self.ln(x)  # training인지 아닌지 적는 것이 없다. ==> LayerNormalization은 batch의 영향을 받지 않기 때문에 traing과 test에 똑같이 동작을 하기 때문.
    return tf.nn.relu(x)


# Define network architecture
class MyModel(tf.keras.Model):  # VGG16 형태
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1_1 = ConvLNReluBlock(16, (3, 3))
        self.conv1_2 = ConvLNReluBlock(16, (3, 3))
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv2_1 = ConvLNReluBlock(32, (3, 3))
        self.conv2_2 = ConvLNReluBlock(32, (3, 3))
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv3_1 = ConvLNReluBlock(64, (3, 3))
        self.conv3_2 = ConvLNReluBlock(64, (3, 3))

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))  # l-2 regularizers 사용 lambda = 0.01 적용
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))

    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비


In [4]:
cifar10 = tf.keras.datasets.cifar10  # 32x32x3  ==>  10 class classification

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(1024)

## Keras API 모델 학습

In [5]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
1563/1563 [==============================] - 18s 11ms/step - loss: 2.2732 - accuracy: 0.3242 - val_loss: 1.4489 - val_accuracy: 0.5060
Epoch 2/100
1563/1563 [==============================] - 17s 11ms/step - loss: 1.3121 - accuracy: 0.5702 - val_loss: 1.1681 - val_accuracy: 0.6247
Epoch 3/100
1563/1563 [==============================] - 17s 11ms/step - loss: 1.1187 - accuracy: 0.6460 - val_loss: 1.0261 - val_accuracy: 0.6842
Epoch 4/100
1563/1563 [==============================] - 17s 11ms/step - loss: 1.0038 - accuracy: 0.6849 - val_loss: 0.9743 - val_accuracy: 0.7010
Epoch 5/100
1563/1563 [==============================] - 17s 11ms/step - loss: 0.9408 - accuracy: 0.7081 - val_loss: 0.9578 - val_accuracy: 0.7023
Epoch 6/100
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8897 - accuracy: 0.7252 - val_loss: 0.9546 - val_accuracy: 0.7045
Epoch 7/100
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8427 - accuracy: 0.7445 - val_loss: 0